In [1]:
import h5py
import glob
import os
import json
import numpy as np
from PIL import Image
import io
import matplotlib.pyplot as plt
import time

In [2]:
def unproject_pixels(pts, cam_matrix, vfov=55, near_plane=0.1, far_plane=100):
    '''
    pts: [N, 2] pixel coords
    depth: [N, ] depth values
    returns: [N, 3] world coords
    '''

    
    camera_matrix = np.linalg.inv(cam_matrix.reshape((4, 4)))

    # Different from real-world camera coordinate system.
    # OpenGL uses negative z axis as the camera front direction.
    # x axes are same, hence y axis is reversed as well.
    # Source: https://learnopengl.com/Getting-started/Camera
    rot = np.array([[1, 0, 0, 0],
                    [0, -1, 0, 0],
                    [0, 0, -1, 0],
                    [0, 0, 0, 1]])
    camera_matrix = np.dot(camera_matrix, rot)
    # print("camera_matrix: ", camera_matrix)
    # print(camera_matrix[1,:-1])

    height = 256#512#depth_map.shape[0]
    width = 256#512#depth_map.shape[1]

    img_pixs = pts[:, [1, 0]].T
    img_pix_ones = np.concatenate((img_pixs, np.ones((1, img_pixs.shape[1]))))

    # Calculate the intrinsic matrix from vertical_fov.
    # Motice that hfov and vfov are different if height != width
    # We can also get the intrinsic matrix from opengl's perspective matrix.
    # http://kgeorge.github.io/2014/03/08/calculating-opengl-perspective-matrix-from-opencv-intrinsic-matrix
    vfov = vfov / 180.0 * np.pi
    tan_half_vfov = np.tan(vfov / 2.0)
    tan_half_hfov = tan_half_vfov * width / float(height)
    fx = width / 2.0 / tan_half_hfov  # focal length in pixel space
    fy = height / 2.0 / tan_half_vfov
    intrinsics = np.array([[fx, 0, width/ 2.0],
                           [0, fy, height / 2.0],
                           [0, 0, 1]])
    img_inv = np.linalg.inv(intrinsics[:3, :3])
    cam_img_mat = np.dot(img_inv, img_pix_ones)

    depth = -camera_matrix[1,-1]/np.dot(cam_img_mat.T, camera_matrix[1,:-1])
    points_in_cam = np.multiply(cam_img_mat, depth)
    points_in_cam = np.concatenate((points_in_cam, np.ones((1, points_in_cam.shape[1]))), axis=0)
    points_in_world = np.dot(camera_matrix, points_in_cam)
    points_in_world = points_in_world[:3, :].T#.reshape(3, height, width)
    points_in_cam = points_in_cam[:3, :].T#.reshape(3, height, width)
    
    return points_in_world

import numpy as np
import matplotlib.pyplot as plt

def get_square(center, num_points, rnge=1):
    '''
    center: [x, z, y]
    '''
    # Define the number of points along each side
    
    center_xy = center[[0, 2]]
    
    bnd = rnge/2

    # Create a grid of x and y values between -1 and 1
    x = np.linspace(-bnd, bnd, num=num_points)
    y = np.linspace(-bnd, bnd, num=num_points)
    X, Y = np.meshgrid(x, y)

    # Create a mask for the points inside the square
    mask = (X >= -1) & (X <= 1) & (Y >= -1) & (Y <= 1)

    # Select the points inside the square
    points = np.column_stack([X[mask], Y[mask]])
    
#     print(center_xy)
#     
    points = points + center_xy[None, :]
    
    points = pad_ones(points)
    
    points[:, 2] = center[1]
    
    points = points[:, [0, 2, 1]]
    
    return points

def get_disc(center, num_points, rnge=1):
    '''
    center: [x, z, y]
    '''
    # Define the number of points along each side
    
    center_xy = center[[0, 2]]
    
#     bnd = rnge/2
   
    # Define the radius of the disc
    radius = rnge

    points = np.random.uniform(low=-2*radius, high=2*radius, size=(num_points, 2))

    # Filter the points to keep only those within the disc
    distances = np.sqrt(np.sum(points**2, axis=1))
    disc_points = points[distances <= radius]
    
    disc_points = disc_points + center_xy[None, :]
    
    disc_points = pad_ones(disc_points)
    
    disc_points[:, 2] = center[1]
    
    disc_points = disc_points[:, [0, 2, 1]]
    
    return disc_points

def filter_occluded(pts, im_seg, color):
    '''
    pts: [N, 2] pixel coords
    returns: [K < N, 2] unoccluded coords
    '''

    target = (im_seg ==  color[None, None, :]).all(-1).astype('float')

    im_seg = (im_seg >0).any(-1).astype('float')

    im_seg = im_seg - target
    
    im_seg = im_seg > 0
    
    im_seg = ~im_seg
    
    valid = im_seg[pts[:, 0], pts[:, 1]]
    
    return pts[valid], im_seg


In [3]:
files = '/ccn2/u/rmvenkat/data/testing_physion/test_contacts_hw_speed/test_humans_consolidated'
original_files = glob.glob(os.path.join(files, '**/**/**/*.hdf5'))

In [4]:
stim_map = '/ccn2/u/thekej/mcvd_physion_readout/test/test_map.json'
with open(stim_map, 'r') as f:
    stimulus = json.load(f)

In [ ]:
def proj_world_to_pixel(pw, cam_matrix, proj_matrix):
    '''
    pw: [N, 3] world coords
    cam_matrix: [4, 4] cam matrix
    proj_matrix: [4, 4] proj matrix
    returns: [N, 2] pixel coords
    '''
    
    matrix = np.matmul(proj_matrix, cam_matrix)
    
    pw = pad_ones(pw).T
    
    proj_pts = np.matmul(matrix, pw).T

    proj_pts = proj_pts/proj_pts[:, 3:4]

    proj_pts = proj_pts.clip(-1, 1)


    proj_pts = (proj_pts + 1)/2
    
    proj_pts = proj_pts[:, :2]

    proj_pts[:, 1] = 1 - proj_pts[:, 1]

    proj_pts = proj_pts[:, [1, 0]]

    proj_pts = (proj_pts*256).astype(int)
    
    return proj_pt

def pad_ones(pts):
    '''
    pts: [N , K]
    returns: [N, K+1]
    '''
    pw = np.concatenate([pts, np.ones([pts.shape[0], 1])], 1)
    return pw

In [ ]:
from scipy import ndimage

def fill_masked_area(mask):
    # Label the connected components in the image
    labeled, num_features = ndimage.label(mask)
    
    # Compute the size of each connected component
    component_sizes = ndimage.sum(mask, labeled, range(num_features + 1))
    
    # Identify the largest connected component
    largest_component = component_sizes.argmax()
    
    # Create a binary mask where the largest component is set to 1 and everything else is 0
    filled_mask = (labeled == largest_component).astype(int)
    
    return filled_mask

In [ ]:
#load hdf5
filename = '/ccn2/u/thekej/R3M_readout/test_feats.hdf5'
with h5py.File(filename, 'r') as f:
    print(f.keys())
    data = f['observed']
    labels = f['label']

In [ ]:
def get_mask(f, f1, frame):
    cam_matrix = f1['frames'][frame]['camera_matrices']['camera_matrix_cam0'][:]
    cam_matrix = cam_matrix.reshape(4, 4)
    proj_matrix = f1['frames'][frame]['camera_matrices']['projection_matrix_cam0'][:]
    proj_matrix = proj_matrix.reshape(4, 4)

    #print(contacts[()])
    contacts = proj_world_to_pixel(np.array(f['frames'][frame]['collisions']['contacts_ot'][()]), cam_matrix, proj_matrix)
    num_samp = 100000

    pts = np.array([[contacts[0][0], contacts[0][1]]])
    #print(pts)
    pw = unproject_pixels(pts, cam_matrix)

    pw_square = get_disc(pw[0], num_samp, rnge=0.2)

    pixel_coord_recon_square = proj_world_to_pixel(pw_square, cam_matrix, proj_matrix)

    mask = np.zeros((256, 256))

    for pt in pixel_coord_recon_square:#_:
        mask[pt[0]:pt[0]+1, pt[1]:pt[1]+1] = 1

    filled = fill_masked_area(mask)
    return filled, pts

In [ ]:
stimuli_names, stimuli_contacts = [], []
counter = 0
masks = []
stimuli_indices = []
for counter, o_f in enumerate(original_files): 
    if counter % 50 == 0:
        print('Done %d out of %d'%(counter, len(original_files)))
    file_name = o_f.split('/')[-3:]
    base_path = ['/ccn2/u/rmvenkat/data/testing_physion/all_final_reproduce_monkey/test_consolidated/lf_0'] + file_name
    file_path = '/'.join(base_path)
    if not os.path.isfile(file_path):
        continue

    stimuli_name = '_'.join(file_path.split('/')[-2:]).replace('.hdf5', '')
    contact_check = False
    if not str(stimuli_name.encode()) in stimulus:
        continue
    with h5py.File(o_f, 'r') as f:
        with h5py.File(file_path, 'r') as f1:
            im_seg = np.array(Image.open(io.BytesIO(f1['frames']['0000']['images']['_id_cam0'][:])))
            frames = list(f['frames'])
            for i, frame in enumerate(frames):
                contacts = f['frames'][frame]['collisions']['contacts_ot']
                if contacts.shape[0] == 0:
                    continue
                mask, pts = get_mask(f, f1, frame)
                contact_check = True
                break
            
            if contact_check:
                stimuli_contacts += [pts]
                stimuli_names += [stimuli_name]
                #stimuli_indices += [stimulus[str(stimuli_name.encode())]]
                masks += [mask]  

In [ ]:
stim_map = '/ccn2/u/thekej/sgnn_features/test_objects_observed_full_outcome__map.json'
with open(stim_map, 'r') as f:
    stimulus = json.load(f)

In [ ]:
masks = np.stack(masks)
stimuli_contacts = np.stack(stimuli_contacts)

stimuli_indices = []

for name in stimuli_names:
    stimuli_indices += [stimulus[str(name)]]

In [ ]:
new_stim_map_file = '/ccn2/u/thekej/placement_task/sgnn_placement/test_observed_full_outcome_map.json'
output_filename = '/ccn2/u/thekej/placement_task/sgnn_placement/test_feats_observed_full_outcome.hdf5'

In [ ]:
new_stimuli_map = {}
for i, stim in enumerate(stimuli_names):
    new_stimuli_map[stim] = i
with open(new_stim_map_file, 'w') as f:
    json.dump(new_stimuli_map, f)

In [ ]:
#load hdf5
filename = '/ccn2/u/thekej/sgnn_features/test_objects_observed_full_outcome.hdf5'

with h5py.File(filename, 'r') as f_in:
    print(f_in.keys())
    data = f_in['features'][:]
    print(data.shape)

In [ ]:
with h5py.File(output_filename, 'w') as f_out:
    # Sort the indices and convert to numpy array
    #sorted_indices = np.sort(indices)

    # Load only the subset of data you need
    subset = data[stimuli_indices]

    # Save the subset to the new HDF5 file
    f_out.create_dataset('features', data=subset)
    f_out.create_dataset('original_indices', data=stimuli_indices)
    f_out.create_dataset('contacts', data=stimuli_contacts)
    f_out.create_dataset('masks', data=masks)

In [ ]:
with h5py.File(output_filename, 'w') as f_out:
    # Sort the indices and convert to numpy array
    #sorted_indices = np.sort(indices)

    # Load only the subset of data you need
    subset = data[stimuli_indices]
    subset1 = data1[stimuli_indices]
    subset2 = data2[stimuli_indices]

    # Save the subset to the new HDF5 file
    f_out.create_dataset('observed', data=subset)
    f_out.create_dataset('observed_full_outcome', data=subset1)
    f_out.create_dataset('simulation', data=subset1)
    f_out.create_dataset('original_indices', data=stimuli_indices)
    f_out.create_dataset('contacts', data=stimuli_contacts)
    f_out.create_dataset('masks', data=masks)